In [1]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np

# corregir puntos de stops

In [9]:
def splitPoint(row):
    row['latitude'] = row['WKT'].split()[2].strip()[:-1]
    row['longitude'] = row['WKT'].split()[1].strip()[1:]
    return row

data_busstops = pd.read_csv('stop_points.csv')
# data_busstops = data_busstops.apply(splitPoint, axis=1)
data_busstops = data_busstops.sort_values(by='number').reset_index(drop=True)
data_busstops = data_busstops.drop(['number', 'WKT'], axis=1) # escoger los paraderos de ida
# data_busstops['id'] = range(36, len(data_busstops)+36)
data_busstops['id'] = data_busstops['id'].apply(lambda id: str(int(id)))
data_busstops['navigation'] = data_busstops['navigation'].values.astype(np.int64)
data_busstops['radio'] = data_busstops['radio'].values.astype(np.int64)

# data_busstops.append(pd.Series([0, 'start', 0, ]))
data_busstops

,id,name,navigation,latitude,longitude,radio
0,0,inicio,159,-13.554978,-71.856025,30
1,36,primer stop,304,-13.554812,-71.857300,18
2,37,segundo stop,281,-13.553294,-71.861944,13
3,38,Ccollana,283,-13.552296,-71.867299,9
4,39,Puente Huaccoto,283,-13.551592,-71.870211,10
...,...,...,...,...,...,...
79,114,Segundo,221,-13.547528,-71.985248,10
80,115,Reservorio,225,-13.549660,-71.988003,10
81,116,Cuarto,230,-13.550408,-71.988880,10
82,117,Quinto,203,-13.550841,-71.989448,10


In [39]:
# df_stops_ida = pd.read_csv('busstops_ida.csv')
# data_busstops['radio'] = df_stops_ida['radio']
# data_busstops

In [10]:
data_busstops.to_csv('busstops_ida.csv')

# Extraer en CSV

In [2]:
# leer dataset monitoring
dt = pd.read_csv('monitoring.csv')
dt['vehicle_id_id'].unique()


# diseño del df: creacion data_time y latitude_longitude
dt['date'] += ' ' + dt['time']
dt['latitude'] = dt.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
dt.drop(['time', 'longitude'], inplace=True, axis=1)
dt = dt.rename(columns={'date':'date_time', 'latitude':'latitude_longitude', 'vehicle_id_id':'vehicle_id'}, inplace=False)
dt = dt.drop_duplicates(subset=['latitude_longitude']) # (65155, 7) (64702, 7)
dt = dt.sort_values(by='date_time')

# convertir de object a datatime
dt['date_time'] = pd.to_datetime(dt['date_time'], format='%Y-%m-%d %H:%M:%S')
print(dt.shape)
dt

(64702, 7)


,id,date_time,lap,latitude_longitude,velocity,navigation,vehicle_id
4824,21937142,2022-07-14 00:01:31,0,"(-13.5537836, -71.8619456)",5,6,SJ37
4978,21937155,2022-07-14 00:03:44,0,"(-13.5540421, -71.8620274)",0,194,SJ37
5039,21937160,2022-07-14 00:05:11,0,"(-13.5383785, -71.9810924)",0,225,SJ39
5061,21937162,2022-07-14 00:05:32,0,"(-13.5538999, -71.8619277)",1,155,SJ37
5081,21937164,2022-07-14 00:05:59,0,"(-13.5476402, -71.8805191)",0,33,SJ07
...,...,...,...,...,...,...,...
62360,22142519,2022-07-14 23:46:49,5,"(-13.5510421, -71.8767183)",0,180,SJ04
62405,22142539,2022-07-14 23:51:05,5,"(-13.5539454, -71.8619251)",2,208,SJ37
62443,22142555,2022-07-14 23:54:43,3,"(-13.5513956, -71.9902558)",0,284,SJ40
62481,22142569,2022-07-14 23:56:00,5,"(-13.5539692, -71.8619186)",0,25,SJ37


In [3]:
dt['vehicle_id_id'].unique()


array(['SJ03', 'SJ07', 'SJ02', 'SJ04', 'SJ20', 'SJ31', 'SJ27', 'SJ29',
       'SJ28', 'SJ10', 'SJ30', 'SJ37', 'SJ23', 'SJ35', 'SJ42', 'SJ36',
       'SJ21', 'SJ38', 'SJ19', 'SJ13', 'SJ39', 'SJ12', 'SJ34', 'SJ18',
       'SJ41', 'SJ08', 'SJ14', 'SJ16', 'SJ09', 'SJ05', 'SJ11', 'SJ01',
       'SJ24', 'SJ40', 'SJ26', 'SJ25', 'SJ06'], dtype=object)

In [50]:
# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    dnav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (dnav<=45 and dis<=stop['radio'])

# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if ( itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            print('F')
            break
    return df.head(i+1)

In [51]:
# parametros de extraccion
veh = 'SJ37'
dt_veh = dt[dt['vehicle_id'] == veh]

dt_vehlap = pd.DataFrame()
laps = list(dt_veh['lap'].unique())[1:-1]
print(list(dt_veh['lap'].unique()), '->', laps)
laps = [3]
for lap in laps:
    dt_vehlap = pd.concat([dt_vehlap, idaPoints(dt_veh[dt_veh['lap']==lap].reset_index(drop=True))])

dt_vehlap


[0, 1, 2, 3, 4, 5] -> [1, 2, 3, 4]
F


,id,date_time,lap,latitude_longitude,velocity,navigation,vehicle_id
0,22002863,2022-07-14 10:46:59,3,"(-13.5546398, -71.8562212)",0,32,SJ37
1,22003604,2022-07-14 10:50:18,3,"(-13.5545592, -71.8560974)",1,110,SJ37
2,22004203,2022-07-14 10:53:06,3,"(-13.5545611, -71.8560971)",2,213,SJ37
3,22004693,2022-07-14 10:55:18,3,"(-13.554613, -71.8561232)",2,200,SJ37
4,22005120,2022-07-14 10:57:22,3,"(-13.55472, -71.8561798)",8,145,SJ37
...,...,...,...,...,...,...,...
229,22023143,2022-07-14 12:16:48,3,"(-13.5517128, -71.9908456)",32,60,SJ37
230,22023201,2022-07-14 12:17:00,3,"(-13.5510793, -71.9897538)",39,52,SJ37
231,22023554,2022-07-14 12:17:49,3,"(-13.5503053, -71.9868593)",25,43,SJ37
232,22023557,2022-07-14 12:18:02,3,"(-13.5506278, -71.9865027)",17,185,SJ37


In [34]:
if len(dt_vehlap['lap'].unique()) != 1:
    name = 'monitoring_veh/' + str(veh) + '.csv'
    dt_vehlap.to_csv(name)
    print(f'print data veh:{veh} laps: {laps[1:-1]} -> {name} len:{dt_vehlap.shape[0]}')
else:
    name = 'monitoring_veh_lap/' + str(veh) + '-lap' + str(dt_vehlap['lap'].unique()[0]) + '.csv'
    dt_vehlap.to_csv(name)
    print(f'print data (veh ,lap) = ({veh}, {laps}) -> {name} len: {dt_vehlap.shape[0]}')

print data (veh ,lap) = (SJ37, [3]) -> monitoring_veh_lap/SJ37-lap3.csv len: 484
